# Workflow for fine Tuning LLM for Multiple Choice | DeBERTaV3 model
**Main Steps:**
* Prepare training & validation data.
* Preporcessing data using Tokenizer from pretrained Model (DeBERTaV3)
* Train the (pretrained) model - incl. the evaluation strategy, metrics
* Generate prediction with new fine-tuned models

Thanks to the [Hugging Face guide for fine-tuning](https://huggingface.co/docs/transformers/tasks/multiple_choice) (which I tried to replicated here)!

# Imports

In [ ]:
!pip install -q accelerate peft==0.5.0 bitsandbytes trl==0.4.7 langchain
!pip install -q --no-cache-dir transformers sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import random

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice, BitsAndBytesConfig, TrainingArguments, Trainer

from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

from sklearn.metrics import accuracy_score

In [ ]:
# Log in to huggingface to load/push fine-tuned models in hub
from huggingface_hub import notebook_login

notebook_login()

# Global Variables

In [ ]:
# model checkpoint
# checkpoint = 'bert-base-uncased'
checkpoint = 'microsoft/deberta-v3-large'
# checkpoint = 'bert-base-uncased'

# multiple choice answers
options = ['A', 'B', 'C', 'D', 'E']

# mapping of answers to integer values
option_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
index_to_option = {0: 'A', 1:'B', 2:'C', 3:'D', 4:'E'}

# 1. Load Datasets

## 1.1. Select Training & Validation  Data

In [ ]:
# Load the training data
data = pd.read_csv('/content/thesis_questions_train.csv').drop(columns="index")
data.reset_index(inplace=True, drop=True)

# Set a random seed for reproducibility
random.seed(42)

# Select 10% of data (randomly) for validation data
validation_data = data.sample(frac=0.10)

# Use the remaining 90% of data for training data
train_data = data.drop(validation_data.index)
# train_data = data

# print training & validation sizes
print(f"Train data: {train_data.shape}, Validation data: {validation_data.shape}")
# print(f"Train data: {train_data.shape}")

Train data: (702, 7), Validation data: (78, 7)


In [ ]:
# examples in train data
train_data.head(5)

,prompt,A,B,C,D,E,answer
0,A potential relationship between a given disea...,cross-sectional study.,case-control study.,ecological study.,correlational study.,descriptive study.,B
1,Which of the following is not an indication fo...,post-inflammatory liver cirrhosis.,Budd-Chiari syndrome.,Wilson’s disease.,primary and secondary biliary liver cirrhosis.,disseminated hepatocellular carcinoma.,E
2,A patient presents to the gynaecological outpa...,overactive bladder.,exercise-induced urinary incontinence caused b...,mixed urinary incontinence.,evacuation of accumulated lymph fluid through ...,vesicovaginal fistula.,E
3,Superior vena cava syndrome:,most commonly occurs in the course of lung can...,may be associated with vessel thrombosis.,is a contraindication to the use of corticoste...,A and B are correct.,"A, B and C are correct.",D
4,The treatment of acute hyperkalemia includes t...,"1,5,6.","2,4,5.","3,5,6.","2,3,6.","1,3,6.",D


## 1.2. Create datasets for training & validation

In [ ]:
# Convert the Pandas dataframe into Dataset
train_ds = Dataset.from_pandas(train_data.reset_index()) #reset_index() since index is randomly out of order
validation_ds = Dataset.from_pandas(validation_data.reset_index())

# print features of train dataset & an example
print(train_ds)
print("An example:\n", train_ds[3])

Dataset({
    features: ['index', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer'],
    num_rows: 702
})
An example:
 {'index': 3, 'prompt': 'Superior vena cava syndrome:', 'A': 'most commonly occurs in the course of lung cancer.', 'B': 'may be associated with vessel thrombosis.', 'C': 'is a contraindication to the use of corticosteroids.', 'D': 'A and B are correct.', 'E': 'A, B and C are correct.', 'answer': 'D'}


# 2. Preprocess
Process the datasets into a format suitable as model's input. From documentation, DeBERTa expects pairs of tokenized sentences and the following fields: 'input_ids', 'token_type_ids', 'attention_mask', 'label'

In [ ]:
# create tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# # test tokenizer
# tokenizer("This is the first sentence.", "This is the second one.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


In [ ]:
def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentences = [example[option] for option in 'ABCDE']

    try:
        tokenized_example = tokenizer(first_sentence, second_sentences, truncation=False)
        tokenized_example['label'] = option_to_index[example['answer']]
        return tokenized_example
    except Exception as e:
        # Print error message and the data causing the error
        print(f"Error during tokenization: {e}")
        print(f"First sentence: {first_sentence}")
        print(f"Second sentences: {second_sentences}")
        # Optional: re-raise the exception or handle it as needed
        raise e


* **Note:** the preprocess function above takes in a single sample for the dataset. It is possible to speed data processing by implement preprocess function that uses multiple samples, which allows use of **batched=True** in map function, and speed up the process.
* Here is the code on Hugging Face: https://huggingface.co/docs/transformers/tasks/multiple_choice

In [ ]:
# tokenize & process datasets for input to model training
tokenized_train_ds = train_ds.map(preprocess, remove_columns=['index', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_validation_ds = validation_ds.map(preprocess, remove_columns=['index', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

# checking output with prints
print(tokenized_train_ds)
print(tokenized_validation_ds)

Map:   0%|          | 0/702 [00:00<?, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 702
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 78
})


# 3. Model Training

* Create a custom data collater for multiple choice training. This helps speed up training by providing dynamic embedding for each training batches --> memory efficient.
* Instruction: https://huggingface.co/docs/transformers/tasks/multiple_choice#train

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True  # Defines how padding should be applied to the inputs
    max_length: Optional[int] = None  # The maximum length of the tokenized inputs
    pad_to_multiple_of: Optional[int] = None  # If set, pads the input to a multiple of this value

    def __call__(self, features):
        # This method is called to create a batch from a list of features (input data) with proper padding
        # The batch consists of samples, where each sample contains all question-answer pairs for a question

        # Determine the correct label key in the features dictionary
        label_name = "label" if "label" in features[0].keys() else "labels"
        # Extract labels from the features and remove the label entries
        labels = [feature.pop(label_name) for feature in features]

        # Get the batch size (number of examples)
        batch_size = len(features)
        # Get the number of choices per question (assuming all questions have the same number of choices)
        num_choices = len(features[0]["input_ids"])

        # Flatten the features for each choice across all examples
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        # Convert the list of lists into a single list
        flattened_features = sum(flattened_features, [])

        # Pad the batch so all inputs are of the same length and convert to tensors.
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Reshape the batch to have dimensions [batch_size, num_choices, sequence_length].
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Convert the list of labels into a tensor and add to the batch.
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)

        # Return the final processed batch ready for input to a model.
        return batch


* Define evaluate metrics for training - using a simple accuracy here.

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = predictions.argmax(axis=1) # Convert continuous values to discrete values
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

* Train the model using Trainer API. For memory constraints context like this, smaller batch_size requires less memory at the cost of longer training time.
* **Note:** choosing training hyper parameters is kinda an art. May re-use the parameters from [Radek's notebook](https://www.kaggle.com/code/radek1/new-dataset-deberta-v3-large-training).

In [ ]:
# load the model
model = AutoModelForMultipleChoice.from_pretrained(checkpoint) # Initialise the pretrained model with a multiple choice classification head

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    warmup_ratio=0.8,
    learning_rate=5e-6,
    # optim='adamw_torch # This is the default optimizer
    evaluation_strategy="epoch", # Comment out if no evaluation on validation set
    # save_strategy="epoch",
    # save_total_limit=1,  # The number of saved checkpoints, can be increased
    per_device_train_batch_size=2, # TRY INCREASE THIS - 2 almost exceeds the RAM limit
    per_device_eval_batch_size=2,
    # gradient_accumulation_steps=2, # Set to 2 to increase the batch size without increasing the memory requirement
    num_train_epochs=4,
    report_to='none',
    push_to_hub=True, # the trained model is pushed to hub. Yayy!
    output_dir='./deberta-v3-lek-train-correct-4epochs',
    # output_dir='./bert-lek-train-4epochs',
    # logging_steps=50,
    # logging_strategy="steps"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_validation_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.609522,0.128205
2,1.615500,1.598319,0.307692
3,1.596700,1.560790,0.294872
4,1.596700,1.493255,0.410256


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

TrainOutput(global_step=1404, training_loss=1.571485774809139, metrics={'train_runtime': 1156.3647, 'train_samples_per_second': 2.428, 'train_steps_per_second': 1.214, 'total_flos': 1616050881128700.0, 'train_loss': 1.571485774809139, 'epoch': 4.0})

* More training probably will help - since we can see loss and accuracy continue to improves after each epoch (on validation set).
* More data probably will help - provided they are relevance to the task (aka medical exam).

# 4. Retrain the model with the full training set

# 5. Predict with new Model
Data needs to be processed the same way like in training, before it can be fed into model for predictions

In [ ]:
# Create Test dataset
test_df = pd.read_csv('/content/thesis_questions_test.csv')
tokenized_test_ds = Dataset.from_pandas(test_df.drop(columns=['index'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E'])

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

In [ ]:
tokenized_test_ds

Dataset({
    features: ['answer', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 195
})

In [ ]:
# Reload the trained BERT/deberta model
# model_trained = AutoModelForMultipleChoice.from_pretrained("yy0514/deberta-v3-lek-train-new-4epochs")

In [ ]:
# reloaded_trainer = Trainer(
#                     model = model_trained,
#                     args=training_args,
#                     tokenizer=tokenizer,
#                     data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
#                     train_dataset=tokenized_train_ds,
#                     # eval_dataset=tokenized_validation_ds,
#                     compute_metrics=compute_metrics,
#                     )

# # predict on test data
# test_preds = reloaded_trainer.predict(tokenized_test_ds)

In [ ]:
# Change column names
top1_col = 'deberta_v3_train_top1'
top3_col = 'deberta_v3_train_top3'
# top1_col = 'bert_train_top1'
# top3_col = 'bert_train_top3'

# predict on test data
test_preds = trainer.predict(tokenized_test_ds)

# converts predictions back to our answer format  - top 3 of (A, B, C, D, E)
preds_as_ids = np.argsort(-test_preds.predictions, 1)
preds_as_answer_letters = np.array(list('ABCDE'))[preds_as_ids]

# add top 1 and top 3 answers to test df
top_1_preds_as_string = test_df[top1_col] = [
    ' '.join(row) for row in preds_as_answer_letters[:, :1]
]

top_3_preds_as_string = test_df[top3_col] = [
    ' '.join(row) for row in preds_as_answer_letters[:, :3]
]
# save the test df
test_df = test_df[['index', 'answer', top1_col, top3_col]]
test_df.to_csv('./test_results.csv', index=False)

# print sample rows in test_df
test_df.head(3)

,index,answer,deberta_v3_train_top1,deberta_v3_train_top3
0,199,B,C,C B D
1,789,E,A,A D C
2,174,B,E,E A C


In [ ]:
# Sense checks
print("Model outputs for 3 test data points:\n ", test_preds.predictions[:3])
print("Sorted outputs as IDs:\n", preds_as_ids[:3])
print("Top 3 outputs as label:\n", top_3_preds_as_string[:3] )

Model outputs for 3 test data points:
  [[-2.4441419  0.4988053  0.7317451 -2.3200583 -2.6143498]
 [ 1.1218463  1.0867928  1.1084112  1.1187598  1.0910959]
 [-1.1337456 -1.5545418 -1.2071853 -1.2173969 -1.1146159]]
Sorted outputs as IDs:
 [[2 1 3 0 4]
 [0 3 2 4 1]
 [4 0 2 3 1]]
Top 3 outputs as label:
 ['C B D', 'A D C', 'E A C']


# Evaluation

In [ ]:
test_df = pd.read_csv("/content/test_results.csv")
test_df.head()

,index,answer,deberta_v3_train_top1,deberta_v3_train_top3
0,199,B,C,C B D
1,789,E,A,A D C
2,174,B,E,E A C
3,467,A,A,A B C
4,66,D,A,A C D


In [ ]:
# Compute the Mean Average Precision (MAP) for the top 3 choices

def precision_at_k(relevance, k):
    assert k <= len(relevance), "k is larger than the length of the list of predictions"
    return sum(relevance[:k]) / k

def MAP_at_3(df, true_col, prediction_col):
    map_score = 0.0
    for _, row in df.iterrows():
        predictions = row[prediction_col].split()  # Split the predictions into a list
        true_answer = row[true_col]
        user_results = [1 if pred == true_answer else 0 for pred in predictions[:3]]

        for k in range(min(len(predictions), 3)):
            # Only consider the first correct label, ignore the rest for MAP calculation
            if user_results[k] == 1:
                map_score += precision_at_k(user_results, k+1)
                break  # Stop after the first correct prediction
    return map_score / len(df)


In [ ]:
def calculate_accuracy(df, true_col, pred_col):
    # Comparing the true answers with the top-1 predictions based on perplexity
    correct_predictions = (df[true_col] == df[pred_col])
    # Calculating the accuracy
    accuracy = correct_predictions.sum() / len(df)
    return accuracy

In [ ]:
# Compute the accuracy
accuracy = calculate_accuracy(test_df, 'answer', top1_col)
print(f"Accuracy: {accuracy}")

# Compute the MAP@3 scores
map_at_3_score = MAP_at_3(test_df, 'answer', top3_col)
print(f"MAP at 3: {map_at_3_score}")

Accuracy: 0.4358974358974359
MAP at 3: 0.6102564102564101
